In [ ]:
import os

In [ ]:
%pwd

'c:\\Users\\artit\\IKP_2025\\fraud_prediction\\research'

In [ ]:
os.chdir("../")

In [ ]:
%pwd

'c:\\Users\\artit\\IKP_2025\\fraud_prediction'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    onedrive_file_path: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from fraud_prediction.constants import *
from fraud_prediction.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            onedrive_file_path=config.onedrive_file_path,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [ ]:
import os
import zipfile
import gdown
from fraud_prediction import logger
from fraud_prediction.utils.common import get_size
from fraud_prediction.config.configuration import ConfigurationManager

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

     
    def download_file(self) -> str:
        try: 
            dataset_url = self.config.onedrive_file_path
            zip_download_dir = self.config.local_data_file
            
            # สร้างไดเรกทอรีปลายทาง
            os.makedirs(os.path.dirname(zip_download_dir), exist_ok=True)
            
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            # วิธีดึง File ID ที่รองรับ URL หลายรูปแบบ
            if "id=" in dataset_url:
                # ถ้าใน YAML เป็น Direct Link อยู่แล้ว ให้ใช้ตัวนั้นเลย
                final_url = dataset_url
            else:
                # ถ้าเป็น View Link ให้ตัดแบ่งเพื่อเอา ID (รองรับทั้งแบบมีและไม่มี / ต่อท้าย)
                url_parts = dataset_url.split("/")
                # โดยปกติ ID จะอยู่หลัง /d/
                if "d" in url_parts:
                    file_id = url_parts[url_parts.index("d") + 1]
                else:
                    file_id = url_parts[-2]
                
                final_url = f'https://drive.google.com/uc?export=download&id={file_id}'

            # ใช้ fuzzy=True เพื่อให้ gdown ช่วยทำความสะอาด URL อีกแรง
            gdown.download(final_url, zip_download_dir, quiet=False, fuzzy=True)

            logger.info(f"Downloaded data successfully into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)




In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-21 17:47:00,288: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-21 17:47:00,288: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-21 17:47:00,294: INFO: common: created directory at: artifacts]
[2026-01-21 17:47:00,296: INFO: common: created directory at: artifacts/data_ingestion]
[2026-01-21 17:47:00,298: INFO: 3277652497: Downloading data from https://drive.google.com/uc?export=download&id=1xHzaCHq6gEXFmYN2Cc2Yb1jVxaAayb2y into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?id=1xHzaCHq6gEXFmYN2Cc2Yb1jVxaAayb2y
To: c:\Users\artit\IKP_2025\fraud_prediction\artifacts\data_ingestion\data.zip
100%|██████████| 7.44k/7.44k [00:00<00:00, 7.39MB/s]

[2026-01-21 17:47:02,169: INFO: 3277652497: Downloaded data successfully into file artifacts/data_ingestion/data.zip]


BadZipFile: File is not a zip file